In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:



def zero_demand_by_minutes(start_dt, end_dt):

    start = datetime.strptime(start_dt, '%Y%m%dT%H:%MZ')
    end = datetime.strptime(end_dt, '%Y%m%dT%H:%MZ')
    time_tup = np.array([start,], dtype='datetime64')
    
    prev_time = start
    while time_tup[-1] < end:
        time_tup = np.append(time_tup, prev_time + timedelta(minutes=1))
        prev_time = prev_time + timedelta(minutes=1)
        #print(time_tup)
    
    zeros = np.zeros(len(time_tup))
    
    #df = pd.DataFrame({'date_time':time_tup, 'demand (watts)':zeros}, index='date_time')
    df = pd.DataFrame({'demand (watts)':zeros}, index=time_tup)
    
    return df


# duration in minutes
def add_demand(df, start_dt, duration, wattage):
    
    start = datetime.strptime(start_dt, '%Y%m%dT%H:%MZ')
    assert(start >= df.index[0] and start <= df.index[-1])    
    
    for i in range(duration):
        df.loc[start+timedelta(minutes=i)] += wattage
    
    return df

def plot_demand(df, title, save_name):
    plt.close()
    fig, ax = plt.subplots(figsize=(8,4))
    #matplotlib.rcParams.update({'font.size': 16})

    ax.set_ylabel('Electricity Demand (Watts)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    plt.title(title)

    #ax.plot(df)
    ax.fill_between(df.index, 0, df['demand (watts)'], color='C1', label='Houshold Demand', alpha=0.7)
    
    ax.xaxis.set_major_locator(mdates.HourLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))#'%Y-%m-%d'))
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=60 )
    #ax.xaxis.set_minor_locator(mdates.DayLocator())

    plt.legend(loc='upper left')
    
    plt.tight_layout()
    plt.grid(axis='y')
    plt.savefig(f"/Users/truggles/blog_plots/vre/{save_name}")
    
    return

    
    
def add_demands(df):
    
    full_day = 60*24
    df = add_demand(df, '20191027T00:00Z', full_day, 150) # vampire
    df = add_demand(df, '20191027T05:45Z', 150, 100) # lights
    df = add_demand(df, '20191027T05:55Z', 5, 2000) # coffee
    df = add_demand(df, '20191027T06:20Z', 5, 1500) # hair dryer
    df = add_demand(df, '20191027T07:20Z', 10, 2000) # eggs
    df = add_demand(df, '20191027T08:15Z', 60, 1800) # dish washer
    
    df = add_demand(df, '20191027T17:00Z', 300, 100) # lights
    df = add_demand(df, '20191027T20:00Z', 45, 900) # laundry
    df = add_demand(df, '20191027T21:00Z', 60, 3000) # cloths dryer
    
    for i in range(0, 24, 5): # compress every 5 hours
        df = add_demand(df, f'20191027T{i}:00Z', 20, 400) # refrigerator compressor
    
    df = add_demand(df, '20191027T18:30Z', 4*60, 1000) # PHEV, 9kWh battery, it takes about 9 hrs to charge
    df = add_demand(df, '20191027T19:30Z', 25, 2000) # dinner
    
    
    return df


def print_integral(df):
    
    tot = 0
    for idx in df.index:
        tot += df.loc[idx]
    print(f"Total integrated energy use: {tot/1000./60} kWh") # / 1000 for kW, /60 b/c this sampled every minute
        
df = zero_demand_by_minutes('20191027T00:00Z', '20191028T00:00Z')
df = add_demands(df)
print_integral(df)
#df

plot_demand(df, 'My Houshold Electricity Usage - Oct 24, 2019', 'my_electricity_use.png')
plot_demand(df, 'My Houshold Electricity Usage - Oct 24, 2019', 'my_electricity_use.pdf')

    
    

In [ ]:
### For EIA usage

# Shows simplified, clean results     
def demand_plot(df, dt_shifted, title, save_name):
    plt.close()
    fig, ax = plt.subplots(figsize=(7,7))
    matplotlib.rcParams.update({'font.size': 16})
    #ax.set_xlabel('Hour')
    ax.set_ylabel('Demand (Megawatts)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    plt.title(title)
    
    
    ax.plot(dt_shifted, df['Cleaned Demand (MW)'], 'C0-', label='Demand', linewidth=3.0)
    
    plt.legend(prop={'size': 20})
    #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.xaxis.set_minor_locator(mdates.HourLocator())
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45 )
    #ax.xaxis.set_minor_locator(mdates.DayLocator())
    ax.set_ylim(min(ax.get_ylim()[0], 0), ax.get_ylim()[1]*1.5)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"/Users/truggles/blog_plots/vre/{save_name}")
    return

for reg, fPath in {'Utilities Commission of New Smyrna Beach': 
                   '~/Downloads/truggles-EIA_Cleaned_Hourly_Electricity_Demand_Data-ac68a5d/data/release_2019_Oct/balancing_authorities/NSB.csv',
                  'Contiguous U.S.': '~/Downloads/truggles-EIA_Cleaned_Hourly_Electricity_Demand_Data-ac68a5d/data/release_2019_Oct/contiguous_US/CONUS.csv'}.items():
    
    df = pd.read_csv(fPath)
    df['date_time'] = pd.to_datetime(df['date_time'])
    dt_shifted = df['date_time'] - timedelta(hours=7)
    demand_plot(df.loc[20000:20072], dt_shifted.loc[20000:20072], reg, fPath.split('/')[-1].replace('.csv','.png'))
    demand_plot(df.loc[20000:20072], dt_shifted.loc[20000:20072], reg, fPath.split('/')[-1].replace('.csv','.pdf'))

In [ ]:
### For solar / wind

# Shows simplified, clean results     
def demand_plot(solar, wind, title, save_name):
    plt.close()
    fig, ax = plt.subplots(figsize=(7,6))
    matplotlib.rcParams.update({'font.size': 16})
    #ax.set_xlabel('Hour')
    ax.set_ylabel('Capacity Factor')
    plt.title(title)
    
    ax.plot(solar.index, solar['solar'], 'C1-', label='Solar', linewidth=3.0)
    ax.plot(solar.index, wind['wind'], 'C2-', label='Wind', linewidth=3.0)
    
    plt.legend(prop={'size': 20})
    #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.xaxis.set_minor_locator(mdates.HourLocator())
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45 )
    #ax.xaxis.set_minor_locator(mdates.DayLocator())
    ax.set_ylim(min(ax.get_ylim()[0], 0), ax.get_ylim()[1]*1.5)
    plt.tight_layout()
    plt.grid()
    plt.savefig(f"/Users/truggles/blog_plots/vre/{save_name}")
    return

def get_df(fName):
    df = pd.read_csv(fName)
    dt = []
    for idx in solar_df.index:
        str_dt = '{}{}{}T{}Z'.format(solar_df.loc[idx, 'Year'], solar_df.loc[idx, 'Month'], 
                                 solar_df.loc[idx, 'Day'], str(solar_df.loc[idx, 'Hour']-1).zfill(2))
        new_dt = datetime.strptime(str_dt, '%Y%m%dT%HZ') - timedelta(hours=7)
        dt.append(new_dt)
    
    df.index = dt
    return df
solar_df = get_df('solar.csv')
wind_df = get_df('wind.csv')
#print(solar_df.head())
#print(wind_df.head())
demand_plot(solar_df.iloc[33:105], wind_df.iloc[33:105], 'Wind and Solar Availability', 'wind_and_solar.png')
demand_plot(solar_df.iloc[33:105], wind_df.iloc[33:105], 'Wind and Solar Availability', 'wind_and_solar.pdf')